In [1]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import ast

## Preprocess Data

In [2]:
import re

def process_triplegs_data(dataframe, time_column='started_at'):
    # Convert to datetime object
    dataframe[time_column] = pd.to_datetime(dataframe[time_column])

    # Define the initial and cutoff dates
    initial_date = pd.to_datetime('1900-01-01').tz_localize('UTC')
    cutoff_date = initial_date + pd.Timedelta(days=30)

    # Filter to within date range
    filtered_data = dataframe.loc[
        (dataframe[time_column] >= initial_date) & (dataframe[time_column] < cutoff_date)
    ].copy()  # .copy() ensures modifications won't trigger a warning

    # Transform LINESTRING into tuples of integer coordinates
    def parse_linestring(linestring):
        # Extract coordinate pairs from the LINESTRING using regex
        coordinates = re.findall(r'(\d+\.\d+ \d+\.\d+)', linestring)
        # Convert the extracted coordinates to integer tuples
        int_coords = [tuple(map(int, map(float, coord.split()))) for coord in coordinates]
        return int_coords  # Return as a list of tuples

    # Apply the transformation to the 'geom' column
    filtered_data['geom'] = filtered_data['geom'].apply(parse_linestring)

    # Group sequences by user ID and collect into lists
    grouped_sequences = filtered_data.groupby('user_id')['geom'].apply(list).tolist()

    # Flatten the nested lists into a single list of coordinate tuples
    all_coordinates = [coords for sequence in grouped_sequences for coords in sequence]

    return all_coordinates

In [3]:
def triplegs_split(df, max_length=500): # Split into 500

    def split_geom_data(geom, max_length):
        points = geom.split(',')
        sub_tpls = [','.join(points[i:i + max_length]) for i in range(0, len(points), max_length)]
        return sub_tpls

    new_rows = []
    for _, row in df.iterrows():
        geoms = split_geom_data(row['geom'], max_length)
        for geom in geoms:
            new_row = row.copy()
            new_row['geom'] = geom
            new_rows.append(new_row)

    return pd.DataFrame(new_rows)

## Apply GSP

In [4]:
from collections import defaultdict
from itertools import combinations

def generate_candidates(sequences, length):
    candidates = set()
    for seq in sequences:
        for i in range(len(seq) - length + 1):
            candidates.add(tuple(seq[i:i + length]))
    return candidates

def is_subsequence(candidate, sequence):
    it = iter(sequence)
    return all(item in it for item in candidate)

def count_support(candidates, sequences):
    support_count = defaultdict(int)
    for candidate in candidates:
        for seq in sequences:
            if is_subsequence(candidate, seq):
                support_count[candidate] += 1
    return support_count


def prune_candidates(support_count, min_support):
    return {seq: count for seq, count in support_count.items() if count >= min_support}

def generate_new_candidates(frequent_sequences, length):
    new_candidates = set()
    frequent_sequences = list(frequent_sequences)
    for seq1, seq2 in combinations(frequent_sequences, 2):
        if seq1[1:] == seq2[:-1]:
            new_candidates.add(seq1 + (seq2[-1],))
    return new_candidates

def gsp(sequences, min_support):
    length = 2
    frequent_sequences = generate_candidates(sequences, length)
    all_frequent_sequences = []

    while frequent_sequences:
        support_count = count_support(frequent_sequences, sequences)
        frequent_sequences = prune_candidates(support_count, min_support)

        all_frequent_sequences.extend(frequent_sequences.keys())

        length += 1
        frequent_sequences = generate_new_candidates(frequent_sequences.keys(), length)

    return all_frequent_sequences


## Save Output to CSV

In [5]:
def gsp_output_save(gsp_output, output_file):
    # Convert sequences to a string representation
    sequences_df = pd.DataFrame({'Sequence': [", ".join(map(str, seq)) for seq in gsp_output]})
    sequences_df.to_csv(output_file, index=False)
    print(f"GSP results saved to {output_file}")


# Data C

In [ ]:
df = pd.read_csv('triplegsC.csv')

# Split long triplegs into shorter sub-triplegs
df = triplegs_split(df)

# Preprocess the data
sequences = process_triplegs_data(df)

In [ ]:
# Use 0.01% of data size as minsup
min_support = int(0.0001 * len(sequences))
print(f"Minimum support threshold: {min_support}")

# Apply GSP
frequent_sequences = gsp(sequences, min_support)


Minimum support threshold: 4


In [11]:
gsp_output_save(frequent_sequences, 'frequent_sequences_C.csv')

GSP results saved to frequent_sequences_C.csv


# Data D

In [ ]:
df = pd.read_csv('triplegsD.csv')

# Split long triplegs into shorter sub-triplegs
df = triplegs_split(df)

# Preprocess the data
sequences = process_triplegs_data(df)

In [ ]:
# Use 0.01% of data size as minsup
min_support = int(0.0001 * len(sequences))
print(f"Minimum support threshold: {min_support}")

# Apply GSP
frequent_sequences = gsp(sequences, min_support)


Minimum support threshold: 4


In [ ]:

gsp_output_save(frequent_sequences, 'frequent_sequences_D.csv')

GSP results saved to frequent_sequences_D.csv


# Data B

In [ ]:
df = pd.read_csv('triplegsB.csv')

# Split long triplegs into shorter sub-triplegs
df = triplegs_split(df)

# Preprocess the data
sequences = process_triplegs_data(df)

In [ ]:
# Use 0.01% of data size as minsup
min_support = int(0.0001 * len(sequences))
print(f"Minimum support threshold: {min_support}")

# Apply GSP
# frequent_sequences = gsp(sequences, min_support)

Minimum support threshold: 4


In [ ]:

gsp_output_save(frequent_sequences, 'frequent_sequences_B.csv')

GSP results saved to frequent_sequences_B.csv
